$15^{th}$ Jul 2021

In [1]:
import pandas as pd
import os, glob
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime

In [2]:
# df = pd.read_excel('/home/douso/Documents/TrendData/Results/COVID-19-Test_result_Reporting_01_01_2021.xlsx', header=1)

In [3]:
cols_of_interest = ['CASE_ID','SAMPLE_NUMBER', 'NAME', 'AGE', 'AGE_UNIT_(DAYS/MONTHS/YEARS)', 'GENDER_(M/F)', 'OCCUPATION', 'NATIONALITY', 
           'COUNTY_OF_RESIDENCE', 'SUB_COUNTY_OF_RESIDENCE', 'HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N', 'TRAVEL_FROM', 
           'QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD', 'SYMTOMS_SHOWN_(COUGH;FEVER;ETC)', 'DATE_OF_SAMPLE_COLLECTION', 
           'DATE_SAMPLE_RECEIVED_IN_THE_LAB', 'RESULT', 'LAB_CONFIRMATION_DATE']

In [4]:
data_dir = '/home/douso/Documents/TrendData/Results'
files = os.listdir(data_dir)
files = sorted(files)

In [5]:
def get_trend_data(df, cols):
    df_new = df.copy()
    df_new.columns = [col.replace(' ', '_').upper() for col in df.columns]
    df_cln = df_new[df_new['SAMPLE_NUMBER'].notna()]
    return df_cln[cols]

In [6]:
# keyerrors = []
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, 'DATE_SAMPLE_RECEIVED_IN_THE_LAB')
# #         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [7]:
# files = ['COVID-19-Test_result_Reporting_01_07_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_01_07_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_08_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_31_07_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_31_07_2020-B.xlsx']

In [8]:
# for file in files:
#     df = pd.read_excel(f'{data_dir}/{file}', header=1)
#     try:
#         df.insert(27, 'DATE SAMPLE RECEIVED IN THE LAB', pd.NaT)
#         df.to_excel(f'/home/douso/Documents/TrendData/Results/add_col/{file}', index=False, startrow=1)
#     except:
#         raise
#         print (f'Error: {file}')
#     finally: pass

In [9]:
# keyerrors = []
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, cols_of_interest)
#         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [10]:
headers = ['TESTING LAB', 'CASE ID', 'TYPE OF CASE (INITIAL/REPEAT)',
       'SAMPLE NUMBER', 'NAME', 'ID/PASSPORT NUMBER', 'AGE',
       'AGE UNIT (DAYS/MONTHS/YEARS)', 'GENDER (M/F)', 'PHONE NUMBER',
       'OCCUPATION', 'NATIONALITY', 'COUNTY OF RESIDENCE',
       'SUB COUNTY OF RESIDENCE', 'VILLAGE/ESTATE OF RESIDENCE', 'WARD',
       'COUNTY OF DIAGNOSIS', 'HAS TRAVEL HOSTORY(LAST 14 DAYS) Y/N',
       'TRAVEL FROM', 'CONTACT WITH CASE Y/N', 'CONFIRMED CASE NAME',
       'QUARANTINE FACILITY/HOSPITAL/HOMESTEAD', 'HAVE SYMPTOMS Y/N',
       'DATE OF ONSET OF SYMPTOMS', 'SYMTOMS SHOWN (COUGH;FEVER;ETC)',
       'SAMPLE TYPE (NP SWAB, OP SWAB, SERUM SPUTUM ETC)',
       'DATE OF SAMPLE COLLECTION', 'DATE SAMPLE RECEIVED IN THE LAB',
       'RESULT', 'LAB CONFIRMATION DATE']

In [11]:
id_headers = ['SAMPLE_NUMBER', 'NAME', 'ID/PASSPORT_NUMBER', 'AGE',
       'AGE_UNIT_(DAYS/MONTHS/YEARS)', 'GENDER_(M/F)', 'PHONE_NUMBER', 'RESULT',
       'LAB_CONFIRMATION_DATE']

In [12]:
# for file in keyerrors[:-1]:
#     df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#     df_file.columns = headers
#     df_file.to_excel(f'{data_dir}/add_col2/{file}', startrow=1)

The following 60 sheets, besides some June sheets (manually fixed earlier), lacked the `DATE_SAMPLE_RECEIVED_IN_THE_LAB` column, it was added for uniformity:

- 'COVID-19-Test_result_Reporting_01_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_01_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_02_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_06_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_06_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_08_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_08_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_09_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_09_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_10_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_11_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_12_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_13_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_13_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_14_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_14_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_15_08_2020-A.xlsx'
- 'COVID-19-Test_result_Reporting_15_08_2020-B.xlsx'
- 'COVID-19-Test_result_Reporting_16_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_16_10_2020 (1).xlsx'
- 'COVID-19-Test_result_Reporting_17_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_17_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_18_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_18_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_19_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_19_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_20_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_20_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_22_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_23_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_08_2020-A.xlsx'
- 'COVID-19-Test_result_Reporting_24_08_2020-B.xlsx'
- 'COVID-19-Test_result_Reporting_25_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_25_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_08_2020.xlsx'

In [13]:
# files = ['COVID-19-Test_result_Reporting_01_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_01_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_02_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_06_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_06_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_08_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_08_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_09_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_09_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_10_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_11_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_12_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_13_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_13_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_08_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_15_08_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_16_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_16_10_2020 (1).xlsx',
#  'COVID-19-Test_result_Reporting_17_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_17_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_19_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_19_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_20_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_20_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_22_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_23_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_08_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_24_08_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_25_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_25_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_08_2020.xlsx']

In [14]:
# for file in files:
#     df = pd.read_excel(f'{data_dir}/{file}', header=1)
#     try:
#         df.insert(27, 'DATE SAMPLE RECEIVED IN THE LAB', np.nan)
#         df.to_excel(f'{data_dir}/add_col/{file}', index=False, startrow=1)
#     except:
#         raise
#         print (f'Error: {file}')
#     finally: pass

The follwing sheets have missing values for the `RESULT` column:
- 'COVID-19-Test_result_Reporting_20_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_16_06_2020.xlsx'

Updated column names of some of the Jun files ['GENDER_(M/F)', 'LAB_CONFIRMATION_DATE', 'RESULT']

In [15]:
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     if file.endswith('.xlsx'):
#         try:
#             df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#             df_data = get_trend_data(df_file, cols_of_interest)
#             df_raw = df_raw.append(df_data)
#             count += 1
#         except: 
#             if KeyError:
# #                 raise
#                 print(f'KeyError: {file}')
#             elif ValueError:
#                 print(f'ValueError: {file}')
#         finally: pass
#     pass
# print(f'Files processed: {count}')
# print(f'Files in the folder: {len(files)}')

### Travel History Missing - the HOSTORY misspelling

- COVID-19-Test_result_Reporting_09_09_2020.xlsx
- COVID-19-Test_result_Reporting_11_09_2020.xlsx
- COVID-19-Test_result_Reporting_12_09_2020.xlsx
- COVID-19-Test_result_Reporting_14_09_2020.xlsx
- COVID-19-Test_result_Reporting_15_09_2020.xlsx
- COVID-19-Test_result_Reporting_18_10_2020.xlsx
- Files processed: 249
- Files in the folder: 256

In [16]:
# keyerrors = []
# # df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, cols_of_interest)
# #         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [17]:
# files = ['COVID-19-Test_result_Reporting_09_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_11_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_12_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_09-2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_10_2020.xlsx']

In [18]:
# for file in files:
#     df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#     (df_file.rename(columns={'HAS TRAVEL HISTORY(LAST 14 DAYS) Y/N': 'HAS TRAVEL HOSTORY(LAST 14 DAYS) Y/N'}, errors='raise').
#     to_excel(f'{data_dir}/rename_col/{file}', index=False, startrow=1))

### Start of find repeat tests

### End of find repeat tests

In [19]:
id_headers = ['SAMPLE_NUMBER', 'NAME', 'AGE',
       'AGE_UNIT_(DAYS/MONTHS/YEARS)', 'GENDER_(M/F)', 'RESULT',
       'LAB_CONFIRMATION_DATE']

In [20]:
df_raw = pd.DataFrame()
count = 0
for file in files:
    if file.endswith('.xlsx'):
        try:
            df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
            df_data = get_trend_data(df_file, cols_of_interest)
            df_raw = df_raw.append(df_data)
            count += 1
        except:
            if KeyError:
                print(f'KeyError: {file}')
# #                 raise
#                 print(f'KeyError: {file}')
            if ValueError:
                print(f'ValueError: {file}')
        finally: pass
    pass
print(f'Files processed: {count}')
print(f'Files in the folder: {len(files)}')

/home/douso/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Files processed: 271
Files in the folder: 273


In [21]:
df_raw.head()

,CASE_ID,SAMPLE_NUMBER,NAME,AGE,AGE_UNIT_(DAYS/MONTHS/YEARS),GENDER_(M/F),OCCUPATION,NATIONALITY,COUNTY_OF_RESIDENCE,SUB_COUNTY_OF_RESIDENCE,HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N,TRAVEL_FROM,QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD,SYMTOMS_SHOWN_(COUGH;FEVER;ETC),DATE_OF_SAMPLE_COLLECTION,DATE_SAMPLE_RECEIVED_IN_THE_LAB,RESULT,LAB_CONFIRMATION_DATE
0,NMS/EM/01F,COVC22771,SAMUEL OYIEKE OKUTHE,58.0,Years,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,NaN,COVC22772,JOSEPH NGANGA MUREGA,63.0,Years,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,NaN,COVC22773,ANN NJERI MUREGA,53.0,Years,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,TL5H/TKA/001,COVC22774,CATHERINE NDUATI,52.0,Years,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,NaN,COVC22775,PERIS KARANI,41.0,Years,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [22]:
os.getenv('PWD')

'/home/douso'

### **Clean Headers**

In [23]:
cols_rename = ['S_NUM', 'AGE', 'AGE_UNIT', 'GEND', 'OCCU', 'NAT', 'COUNT_RES', 'S_COUNT_RES', 'TRAV_HIST', 
                    'TRAV_FROM', 'QUAR_PLACE', 'SYMPS', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'RESULT', 'DT_CONF']

In [24]:
# df_raw.columns = cols_rename

### **Clean Sample Numbers**

In [25]:
def cov_names(x):
    x = x.upper().replace(' ', '')
    if len(x) == 8:
        x = x.replace('COVC', 'COVC0')
    elif len(x) == 7:
        x = x.replace('COVC', 'COVC00')
    elif len(x) == 6:
        x = x.replace('COVC', 'COVC000')
    elif len(x) == 5:
        x = x.replace('COVC', 'COVC0000')
    return x

In [26]:
df_raw.head()

,CASE_ID,SAMPLE_NUMBER,NAME,AGE,AGE_UNIT_(DAYS/MONTHS/YEARS),GENDER_(M/F),OCCUPATION,NATIONALITY,COUNTY_OF_RESIDENCE,SUB_COUNTY_OF_RESIDENCE,HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N,TRAVEL_FROM,QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD,SYMTOMS_SHOWN_(COUGH;FEVER;ETC),DATE_OF_SAMPLE_COLLECTION,DATE_SAMPLE_RECEIVED_IN_THE_LAB,RESULT,LAB_CONFIRMATION_DATE
0,NMS/EM/01F,COVC22771,SAMUEL OYIEKE OKUTHE,58.0,Years,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,NaN,COVC22772,JOSEPH NGANGA MUREGA,63.0,Years,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,NaN,COVC22773,ANN NJERI MUREGA,53.0,Years,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,TL5H/TKA/001,COVC22774,CATHERINE NDUATI,52.0,Years,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,NaN,COVC22775,PERIS KARANI,41.0,Years,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [27]:
cols_of_interest2 = ['CASE_ID', 'S_NUM', 'NAME', 'AGE', 'AGE_UNIT', 'GEND', 'OCCU', 'NAT', 'COUNT_RES', 'S_COUNT_RES', 'TRAV_HIST', 
                    'TRAV_FROM', 'QUAR_PLACE', 'SYMPS', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'RESULT', 'DT_CONF']

In [28]:
df_raw.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx', 
              index=False, header=cols_of_interest2)

In [29]:
# df_raw.head()

In [30]:
# df_mrg_mtdata = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx')

In [31]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] != 'Years'].AGE_UNIT.unique()

In [32]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] == 'Months']#.AGE_UNIT.unique()

In [33]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] == 'Days']#.AGE_UNIT.unique()

In [34]:
df_mrg_mtdata = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx', usecols=cols_rename)

### **Clean Sample Name**

In [35]:
df_name = df_mrg_mtdata.assign(S_NUM=df_mrg_mtdata['S_NUM'].apply(lambda x: cov_names(x)))

In [36]:
df_name[df_name['S_NUM'].str.len() < 8]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9607,COVE20,39,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
9608,COVE21,50,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
17414,NIL,40,Years,M,Nil,Kenya,Laikipia,Laikipia East,No,NaN,NTRH,NaN,2020-06-18 00:00:00,NaN,Negative,2020-06-20 00:00:00
25157,KEMRI-1,NaN,Years,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Positive,19-04-2021
25161,KEMRI-2,NaN,Years,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Positive,19-04-2021
25162,KEMRI-3,NaN,Years,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Positive,19-04-2021
25163,KEMRI-4,NaN,Years,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Positive,19-04-2021
25164,KEMRI-5,NaN,Years,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Positive,19-04-2021
25165,KEMRI-6,NaN,Years,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Positive,19-04-2021
25166,KEMRI-7,NaN,Years,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Positive,19-04-2021


In [37]:
df_name[df_name['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9607,COVE20,39,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
9608,COVE21,50,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
17414,NIL,40,Years,M,Nil,Kenya,Laikipia,Laikipia East,No,NaN,NTRH,NaN,2020-06-18 00:00:00,NaN,Negative,2020-06-20 00:00:00
24773,DRC-BKV-70,39,Years,M,NaN,Democratic Republic of the Congo,Bukavu,NaN,NaN,NaN,NaN,CO;RN;LT;GW,01-06-2020,02-08-2021,Positive,NaN
24774,DRC-BKV-53,30,Years,M,NaN,Democratic Republic of the Congo,Bukavu,NaN,NaN,NaN,NaN,FC;SB;CO;GW,01-06-2020,02-08-2021,Positive,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25216,NHRL-S033,26,Years,F,NaN,Kenya,Nairobi,NaN,No,NaN,NaN,NaN,23/4/2021,NaN,Positive,25/4/2021
25217,NHRL-S034,45,Years,F,NaN,Kenya,Nairobi,NaN,No,NaN,NaN,NaN,23/4/2021,NaN,Positive,25/4/2021
25218,NHRL-S035,30,Years,F,NaN,Kenya,Nairobi,NaN,No,NaN,NaN,NaN,23/4/2021,NaN,Positive,25/4/2021
25219,NHRL-S036,30,Years,M,NaN,Kenya,Nairobi,NaN,No,NaN,NaN,NaN,23/4/2021,NaN,Positive,25/4/2021


### **Clean Age**

In [38]:
df_name[df_name['AGE_UNIT'] == 'Days']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [39]:
df_months = df_name[df_name['AGE_UNIT'] == 'Months']

In [40]:
df_months.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
581,COVC09555,11,Months,F,NaN,Kenya,Narok,RONGAI,No,NaN,NaN,NaN,2020-09-01 00:00:00,NaN,Negative,2020-09-02 00:00:00
1206,COVC02581,1,Months,M,Inmate,Uganda,Kiambu,Thika,N,NIL,Prison,NIL,2020-07-01 00:00:00,NaN,Negative,2020-07-03 00:00:00
1634,COVC17631,10,Months,F,NaN,Kenya,Kiambu,KIKUYU,No,NaN,NaN,NaN,2020-10-27 00:00:00,2020-10-03 00:00:00,Negative,2020-11-03 00:00:00
2485,COVC18026,6,Months,M,NaN,Kenya,Kiambu,THIKA,No,NaN,NaN,CO,2020-11-03 00:00:00,2020-11-04 00:00:00,Positive,2020-11-04 00:00:00
2642,COVC18183,19,Months,M,Child,Kenya,Kiambu,LIMURU,No,NaN,NaN,NaN,2020-11-03 00:00:00,2020-11-04 00:00:00,Negative,2020-11-04 00:00:00


In [41]:
df_mn2yrs = df_months.assign(AGE=df_months['AGE'].map(lambda x: round(x / 12, 1))).replace('Months', 'Years')

In [42]:
df_mn2yrs.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
581,COVC09555,0.9,Years,F,NaN,Kenya,Narok,RONGAI,No,NaN,NaN,NaN,2020-09-01 00:00:00,NaN,Negative,2020-09-02
1206,COVC02581,0.1,Years,M,Inmate,Uganda,Kiambu,Thika,N,NIL,Prison,NIL,2020-07-01 00:00:00,NaN,Negative,2020-07-03
1634,COVC17631,0.8,Years,F,NaN,Kenya,Kiambu,KIKUYU,No,NaN,NaN,NaN,2020-10-27 00:00:00,2020-10-03 00:00:00,Negative,2020-11-03
2485,COVC18026,0.5,Years,M,NaN,Kenya,Kiambu,THIKA,No,NaN,NaN,CO,2020-11-03 00:00:00,2020-11-04 00:00:00,Positive,2020-11-04
2642,COVC18183,1.6,Years,M,Child,Kenya,Kiambu,LIMURU,No,NaN,NaN,NaN,2020-11-03 00:00:00,2020-11-04 00:00:00,Negative,2020-11-04


In [43]:
df_name[df_name['AGE_UNIT'] == 'Days']#.AGE_UNIT.unique()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [44]:
df_not_months = df_name[df_name['AGE_UNIT'] != 'Months']

In [45]:
df_not_months.shape

(25144, 16)

In [46]:
df_mn2yrs.shape

(77, 16)

In [47]:
df_years = df_not_months.append(df_mn2yrs)

In [48]:
df_years.shape

(25221, 16)

In [49]:
df_years['AGE_UNIT'].unique()

array(['Years', nan, 'years', 'Nil'], dtype=object)

In [50]:
df_years[df_years['AGE_UNIT'] == 'NO PATIENT DATASHEET']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [51]:
df_years[df_years['AGE_UNIT'].str.contains('M', 'F') == True]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [52]:
df_years[df_years['AGE_UNIT'] == 'Nil']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
14979,COVC04083,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14980,COVC04084,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14981,COVC04085,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Positive,2020-07-17 00:00:00
14982,COVC04086,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14983,COVC04087,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14984,COVC04088,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14985,COVC04089,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14986,COVC04090,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14987,COVC04091,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14988,COVC04092,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00


In [53]:
df_years_drop_age_unit = df_years.drop('AGE_UNIT', axis=1)

In [54]:
df_years_rencol = df_years_drop_age_unit.rename(columns={'AGE': 'AGE_YRS'})

In [55]:
df_years_rencol.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [56]:
df_years_rencol['AGE_YRS'].unique()

array([58, 63, 53, 52, 41, 9, 6, 39, 23, 36, 33, 42, 34, 32, 57, 56, 27,
       20, nan, 44, 26, 31, 35, 28, 37, 54, 29, 43, 24, 38, 47, 49, 40,
       51, 50, 25, 30, 45, 46, 22, 21, 48, 55, 65, 64, 60, 18, 16, 10, 17,
       15, 8, 13, 5, 12, 84, 59, 62, 68, 312, 61, 19, 2, 86, 80, 69, 72,
       11, 70, 88, 3, 4, 7, 74, 67, 87, 3.6, 1, 14, 1.5, 2.42,
       'None indicated', 73, 6.5, 2.5, 75, 97, 'Nil', 71, 89, 96, 82, 77,
       66, 'Not indicated', 743, 78, 76, 'None', 9.5, 'nil', 79, 81, 98,
       'none indicated', 90, 85, 91, 100, 101, 83, 92, 'not_indicated',
       5.5, 'Not Indicated', '37', '36', '40', '38', '29', 94, 105, 0.5,
       'not Indicated', '30', '44', '33', '27', '26', '51', '28',
       'None Indicated', 542, 0.8, 0.9, 0.1, 1.6, 1.1, 2.9, 3.7, 0.2, 1.7,
       0.3, 2.8, 0.7, 1.2, 2.1, 0.4, 4.5, 1.8, 0.6], dtype=object)

In [57]:
[x for x in df_years_rencol['AGE_YRS'].unique() if isinstance(x, str)]

['None indicated',
 'Nil',
 'Not indicated',
 'None',
 'nil',
 'none indicated',
 'not_indicated',
 'Not Indicated',
 '37',
 '36',
 '40',
 '38',
 '29',
 'not Indicated',
 '30',
 '44',
 '33',
 '27',
 '26',
 '51',
 '28',
 'None Indicated']

In [58]:
to_rep = ['None indicated',
 'Nil',
 'Not indicated',
 'None',
 'nil',
 'none indicated',
 'not_indicated',
 'Not Indicated',
 'not Indicated',
 'None Indicated',
 'NO PATIENT DATASHEET'
]

In [59]:
df_years_repnan2float = df_years_rencol.assign(AGE_YRS=df_years_rencol['AGE_YRS'].replace(to_rep, np.nan).astype(float))

In [60]:
df_years_repnan2float

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22576,COVC16100,1.0,F,NaN,Kenya,Nakuru,GILGIL,No,NaN,NaN,NaN,2020-10-23 00:00:00,2020-10-24 00:00:00,Negative,2020-10-28 00:00:00
23102,COVC16315,0.9,M,NaN,Kenya,Laikipia,LAIKIPIA WEST,No,NaN,NaN,FC;CO;D,2020-10-25 00:00:00,2020-10-27 00:00:00,Negative,2020-10-29 00:00:00
23671,COVC16833,0.8,M,NaN,Kenya,Nairobi,DONHOLM,No,NaN,NaN,NaN,2020-10-29 00:00:00,2020-10-29 00:00:00,Negative,2020-10-30 00:00:00
23927,COVC17089,0.6,M,NaN,Kenya,Mombasa,LIKONI,No,NaN,NaN,CO,2020-10-22 00:00:00,2020-10-27 00:00:00,Negative,2020-10-30 00:00:00


### **Clean Gender**

In [61]:
df_years_repnan2float['GEND'].unique()

array(['M', 'F', 'f', 'Female', 'Male', 'F`', 'MALE ', 'MALE', nan, 'Nil',
       'T'], dtype=object)

In [62]:
df_years_repnan2float[df_years_repnan2float['GEND'].isna()].sort_values('S_NUM')

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9988,COVC00749,42.0,NaN,Nil,Kenyan,Laikipia,Laikipia East,N,NaN,Nanyuki Teaching and Referral Hospital,Asymtomatic,9th June 2020,"June 11, 2020",Neg,2020-06-12 00:00:00
13009,COVC01037,30.0,NaN,Nil,Kenyan,Nairobi,Embaksi,N,NaN,Nil,Asymtomatic,Nil,2020-06-15 00:00:00,Neg,2020-06-15 00:00:00
18488,COVC04494,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
18489,COVC04495,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
18490,COVC04496,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25176,NPHL-12780,NaN,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,07-04-2021,Positive,19-04-2021
25180,NPHL1,NaN,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,07-04-2021,Positive,19-04-2021
25181,NPHL2,NaN,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,07-04-2021,Positive,19-04-2021
25182,NPHL3,NaN,NaN,NaN,Kenya,NaN,NaN,No,NaN,NaN,NaN,NaN,07-04-2021,Positive,19-04-2021


In [63]:
df_years_repnan2float[df_years_repnan2float['GEND'] == 'Nil']

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
11365,COVC00749,42.0,Nil,Nil,Kenyan,Laikipia,Laikipia East,N,NaN,Nanyuki Teaching and Referral Hospital,Asymtomatic,9th June 2020,"June 11, 2020",Neg,2020-06-12 00:00:00
16507,COVC01628,NaN,Nil,Nil,Nil,Nyeri,NaN,No,NaN,NaN,NaN,NaN,NaN,ms,2020-06-19 00:00:00


COVEs:
- COVC13844
- COVC13845
- COVC13846
- COVC13847
- COVC13848
- COVC13849
- COVC13850
- COVC13851
- COVC13852
- COVC13853
- COVC16256
- COVC16257

In [64]:
df_years_repgender = df_years_repnan2float.assign(GEND=df_years_repnan2float['GEND'].replace(['f', 'Female', 'F`', 'T'], 'F')
                                                 .replace(['Male', 'MALE ', 'MALE'], 'M'))

In [65]:
df_years_repgender['GEND'].unique()

array(['M', 'F', nan, 'Nil'], dtype=object)

In [66]:
df_years_repgender['GEND'].replace('Nil', np.nan, inplace=True)

In [67]:
df_years_repgender['GEND'].unique()

array(['M', 'F', nan], dtype=object)

### **Clean Occupation**

In [68]:
df_years_repgender['OCCU'].unique()

array(['HCW', 'CEC', 'Psychologist', ..., 'Logistician', 'Hotel',
       'Child accompanying inmate mothe'], dtype=object)

In [69]:
df_years_repgender['OCCU'] = df_years_repgender['OCCU'].replace(['Nil','nil'], np.nan)

### **Clean Nationality**

In [70]:
df_years_repgender['NAT'].unique()

array(['Kenya', 'Ethiopia', 'Somalia', 'Uganda', 'South Africa', 'India',
       'United States of America (USA)', 'Germany', 'Sudan',
       'South Sudan', nan, 'Lebanon', 'Sweden', 'Nigeria', 'Benin',
       'United Kingdom (UK)', 'Netherlands', 'Cuba', 'Kenyan', 'Indian',
       'Ghana', 'Zimbabwe', 'Rwanda', 'Mauritania', 'Denmark', 'Pakistan',
       'Tanzania', 'Canada', 'Congo, Democratic Republic of the',
       'Burundi', 'Italy', 'Australia', 'Colombia', 'Hungary', 'Djibouti',
       'Finland', 'Senegal', 'kenya', 'Yemen', 'Turkey', 'Philippines',
       'Japan', 'Ireland', 'Tunisia', 'Switzerland', 'Togo',
       'Burkina Faso', 'Brazil', 'Portugal', 'Cameroon', 'Bangladesh',
       'Madagascar', "Cote d'Ivoire", 'Egypt', 'China', 'KENYAN',
       'Greece', 'Bulgaria', 'Nil', 'Mozambique', 'Mali', 'Eritrea',
       'Austria', 'Malawi', 'France', 'Namibia', 'Congo, Republic of the',
       'Zambia', 'Sri Lanka', 'United States', 'Jordan',
       'Democratic Republic of the Co

In [71]:
df_years_repnat = df_years_repgender.assign(NAT=df_years_repgender['NAT'].replace('Nil', np.nan)
                                            .replace(['United States', 'Indian'
                                                      , 'Congo, Democratic Republic of the'
                                                     , 'Congo, Republic of the'] 
                                                     , ['United States of America (USA)','India'
                                                        , 'Democratic Republic of the Congo',
                                                       'Republic of the Congo'])
                                            .map(lambda x: str(x).replace(x, 'Kenya') if ('enya' in str(x).lower()) else x))

In [72]:
df_years_repnat['NAT'].unique()

array(['Kenya', 'Ethiopia', 'Somalia', 'Uganda', 'South Africa', 'India',
       'United States of America (USA)', 'Germany', 'Sudan',
       'South Sudan', nan, 'Lebanon', 'Sweden', 'Nigeria', 'Benin',
       'United Kingdom (UK)', 'Netherlands', 'Cuba', 'Ghana', 'Zimbabwe',
       'Rwanda', 'Mauritania', 'Denmark', 'Pakistan', 'Tanzania',
       'Canada', 'Democratic Republic of the Congo', 'Burundi', 'Italy',
       'Australia', 'Colombia', 'Hungary', 'Djibouti', 'Finland',
       'Senegal', 'Yemen', 'Turkey', 'Philippines', 'Japan', 'Ireland',
       'Tunisia', 'Switzerland', 'Togo', 'Burkina Faso', 'Brazil',
       'Portugal', 'Cameroon', 'Bangladesh', 'Madagascar',
       "Cote d'Ivoire", 'Egypt', 'China', 'Greece', 'Bulgaria',
       'Mozambique', 'Mali', 'Eritrea', 'Austria', 'Malawi', 'France',
       'Namibia', 'Republic of the Congo', 'Zambia', 'Sri Lanka',
       'Jordan'], dtype=object)

### **Clean County of Res**

In [73]:
df_years_repnat.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [74]:
len(df_years_repnat['S_COUNT_RES'].str.lower().unique())

923

In [75]:
len((df_years_repnat['S_COUNT_RES']
     .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).strip().capitalize())
     .unique()))

844

In [76]:
s_counties = (df_years_repnat['S_COUNT_RES']
     .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).replace(str(x), str(np.nan)) if ('indicate' in str(x)) else str(x).strip().capitalize())
              .replace('nan', np.nan))

In [77]:
# sorted([s for s in s_counties.unique()])

In [78]:
counties = (df_years_repnat['COUNT_RES']
                   .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).strip().capitalize() if (isinstance(x, str)) else x))

In [79]:
counties.unique()

array(['Nairobi', 'Kiambu', 'Kajiado', 'Machakos', 'Laikipia', 'Marsabit',
       'Makueni', "Murang'a", 'Isiolo', 'Garissa', 'Homabay', 'Nakuru',
       'Samburu', 'Mombasa', nan, 'Nyeri', 'Kisii', 'Tana river', 'Kitui',
       'Wajir', 'Nyandarua', 'Kirinyaga', 'Meru', 'Uasin gishu', 'Siaya',
       'Bungoma', 'Kakamega', 'Nandi', 'Kilifi', 'Kwale', 'Muranga',
       'Uganda', 'Mandera', 'Kampala', 'Niarobi', 'Kisumu', 'Badiere',
       'Embu', 'Busia', 'Kigali', 'Mukono/uganda', 'Congo', 'Nil',
       'Vihiga', 'Narok', 'Migori', 'Kericho', 'Trans nzoia',
       'Elgeyo marakwet', 'Kajido', 'Taita taveta', 'Bomet', 'Nyamira',
       'Bukavu', 'Thika'], dtype=object)

In [80]:
df_years_county = (df_years_repnat.assign(COUNT_RES=counties)).assign(S_COUNT_RES=s_counties)

In [81]:
df_years_county['COUNT_RES'].unique()

array(['Nairobi', 'Kiambu', 'Kajiado', 'Machakos', 'Laikipia', 'Marsabit',
       'Makueni', "Murang'a", 'Isiolo', 'Garissa', 'Homabay', 'Nakuru',
       'Samburu', 'Mombasa', nan, 'Nyeri', 'Kisii', 'Tana river', 'Kitui',
       'Wajir', 'Nyandarua', 'Kirinyaga', 'Meru', 'Uasin gishu', 'Siaya',
       'Bungoma', 'Kakamega', 'Nandi', 'Kilifi', 'Kwale', 'Muranga',
       'Uganda', 'Mandera', 'Kampala', 'Niarobi', 'Kisumu', 'Badiere',
       'Embu', 'Busia', 'Kigali', 'Mukono/uganda', 'Congo', 'Nil',
       'Vihiga', 'Narok', 'Migori', 'Kericho', 'Trans nzoia',
       'Elgeyo marakwet', 'Kajido', 'Taita taveta', 'Bomet', 'Nyamira',
       'Bukavu', 'Thika'], dtype=object)

### **Clean Subcounty of Res

In [82]:
df_years_scounty = df_years_county.assign(S_COUNT_RES=s_counties)

### **Clean Travel History**

In [83]:
df_years_scounty.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [84]:
df_years_scounty['TRAV_HIST'].unique()

array(['No', nan, 'Yes', 'N', 'JH', 'no', 'NO ', 'Nil', 'YES', 'NONE ',
       'NONE', 'Y', 'y', 'yes'], dtype=object)

In [85]:
df_travl = df_years_scounty.assign(TRAV_HIST=df_years_scounty['TRAV_HIST'].replace([np.nan, 'N', 'no', 'NO', 'Nil', 'NONE ', 'NONE'], 'No')
           .replace(['YES', 'Y', 'y', 'yes'], 'Yes'))


In [86]:
df_travl['TRAV_FROM'].unique()

array([nan, 'ELDORET', 'Nairobi', 'Kendu-Bay;Kisii;Kisumu;Homabay;Oyugis',
       'Mumias, Oyugis, Homabay', 'NAIVASHA', 'NAIROBI;NYERI;NANYUKI',
       'NIL', 'Ruaka', 'Ngoingwa', 'Limuru', 'Thika', 'Embakasi',
       'Uganda', 'Dubai', 'Nil', 'Not indicated on form ',
       'TORORO UGANDA', 'UGANDA', 'AWASI', 'KIGALI', 'ENTEBE', 'NAIROBI',
       'JINJA', 'KISUMU', 'KAMPALA', 'KASESE', 'KILIFI COUNTY',
       'KWALE COUNTY', 'KENDU BAY', 'KISUMU, KITALE', 'HOMABAY',
       'KISII, KISUMU', 'HOMABAY, KISII', 'NYAHURURU; NANYUKI', 'TORORO',
       'KAMPALA UGANDA', 'RWANDA', 'JINJA UGANDA', 'KAJIADO', 'TANZANIA',
       'NOT APPLICABLE ', 'SOUTH SUDAN ', 'NYERI', 'KWALE;KILIFI',
       'KAKAMEGA', 'KWALE', 'Mombasa', 'Eldoret', 'Malindi',
       'Uasin Gishu', 'N', 'Machakos', 'No', 'KILIFI/NAIROBI',
       'NANYUKI SAGANA', 'THARAKA NITHI', 'SHIMBA HILLS', 'KILIFI',
       'MACHAKOS', 'Netherlands', 'MIGORI', 'Nairobi;USA', 'USA',
       'Homa-Bay;Kisumu;Nairobi;Kerugoya-Kutus', 'TAV

In [87]:
df_travl2 = (df_travl.assign(TRAV_FROM=df_travl['TRAV_FROM'].replace(['nan','NIL', 'Nil', 'Not indicated on form '
                  , 'NOT APPLICABLE ', 'N', 'N', 'No', 'not_indicated', 'Not applicable', 'Not indicated'],np.nan)
                    .replace(['Mombasa (stayed in mtwapa for 4 days)', 'KINANGO MSAMBWENI', 'Nan'
                              , 'Mombasa (Stayed in Mtwapa for 4 days)'], 
                    ['Mtwapa', 'Kinangop;Msambweni', np.nan, 'Mtwapa'])
                    .apply(lambda x: str(x).replace('/', ';')
                    .replace(' and ', ';')
                    .replace('-', ';')
                    .replace(',', ';')
                    .replace(' ; ', ';')
                    .replace('; ', ';')
                    .strip().capitalize() if (isinstance(x, str)) else x)))

In [88]:
df_travl2['TRAV_FROM'].unique()



array([nan, 'Eldoret', 'Nairobi', 'Kendu;bay;kisii;kisumu;homabay;oyugis',
       'Mumias;oyugis;homabay', 'Naivasha', 'Nairobi;nyeri;nanyuki',
       'Ruaka', 'Ngoingwa', 'Limuru', 'Thika', 'Embakasi', 'Uganda',
       'Dubai', 'Tororo uganda', 'Awasi', 'Kigali', 'Entebe', 'Jinja',
       'Kisumu', 'Kampala', 'Kasese', 'Kilifi county', 'Kwale county',
       'Kendu bay', 'Kisumu;kitale', 'Homabay', 'Kisii;kisumu',
       'Homabay;kisii', 'Nyahururu;nanyuki', 'Tororo', 'Kampala uganda',
       'Rwanda', 'Jinja uganda', 'Kajiado', 'Tanzania', 'South sudan',
       'Nyeri', 'Kwale;kilifi', 'Kakamega', 'Kwale', 'Mombasa', 'Malindi',
       'Uasin gishu', 'Machakos', 'Kilifi;nairobi', 'Nanyuki sagana',
       'Tharaka nithi', 'Shimba hills', 'Kilifi', 'Netherlands', 'Migori',
       'Nairobi;usa', 'Usa', 'Homa;bay;kisumu;nairobi;kerugoya;kutus',
       'Taveta', 'Mtwapa', 'Kinangop;msambweni', 'Meru', 'Muhonia',
       'Kasese uganda', 'Kirinyaga', 'Kiambu', 'Dar es saalam',
       'Taita 

In [89]:
df_travl2.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


### **Clean Symptoms**

- GW: General weakness
- FC: Fever/Chills
- CO: Cough
- ST: Sore throat
- RN: Runny nose
- SB: Shortness of breath/Difficulty in breathing
- D: Diarrhoea
- NV: Nausea/Vomiting
- H: Headache
- IC: Irritability/Confusion
- P: Pain
- LS: Loss of smell
- LT: Loss of taste
- P-M: Muscular pain
- P-A: Abdominal pain
- P-C: Chest pain
- P-J: Joint pain

In [90]:
df_travl2['SYMPS'] = (df_travl2['SYMPS'].apply(lambda x: str(x).replace(str(x), str(np.nan)) if ('symtom' in str(x)) else str(x).replace('F/C', 'FC'))
                      .replace(['[=]','nan'], np.nan))

### **Clean Dates**

In [91]:
def my_date_parser(x):
    try:
        if type(x) == pd.Timestamp:
            x = x#.strftime('%d-%m-%Y')
        if type(x) != pd.Timestamp and type(x) != (int, float, pd.NaT):
            x = pd.to_datetime(x, errors='ignore', dayfirst=True)#.strftime('%d-%m-%Y')
        if x in ['None indicated', 'NIL', 'Not indicated on form ',
                    'Nil', 'None indicated', 'Leaked Sample - Empty', 'nan']:
            x = pd.NaT
    except (ValueError, AttributeError):
        x = pd.NaT
    return x

In [92]:
df_dt1 = df_travl2.assign(DT_SAM_COLL=df_travl2['DT_SAM_COLL'].apply(lambda x: my_date_parser(x)))

In [93]:
df_dt2 = df_dt1.assign(DT_SAM_RECEP=df_dt1['DT_SAM_RECEP'].apply(lambda x: my_date_parser(x)))

In [94]:
df_dts_cln = df_dt2.assign(DT_CONF=df_dt2['DT_CONF'].apply(lambda x: my_date_parser(x)))

In [95]:
df_dts_cln.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,2021-01-01,2021-01-01,Negative,2021-01-01
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,2020-12-31,2021-01-01,Negative,2021-01-01
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,NaN,NaN,NaN,2021-01-01,2021-01-01,Negative,2021-01-01
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,NaN,NaN,CO;DIB,2021-01-02,2021-01-01,Negative,2021-01-01
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,NaN,NaN,CO;DIB,2021-01-03,2021-01-01,Positive,2021-01-01


### **Clean Results**

In [96]:
df_dts_cln['RESULT'].unique()

array(['Negative', 'Positive', 'neg', 'pos', 'ms', 'negative', 'positive',
       'Neg', 'Pos', 'MS', 'Positve', 'Positive ', nan, 'POSITIVE'],
      dtype=object)

In [97]:
df_dts_cln['RESULT'].map(lambda x: x.strip().lower() if (isinstance(x, str)) else x).unique()

array(['negative', 'positive', 'neg', 'pos', 'ms', 'positve', nan],
      dtype=object)

In [98]:
def cln_results(x):
    if isinstance(x, str):
        x = x.strip().lower()
        if 'neg' in x:
            x = x.replace(x, 'Negative')
        elif 'pos' in x:
            x = x.replace(x, 'Positive')
        elif x == 'inconclusive':
            x = x.replace(x, 'Negative')
    return x
        

In [99]:
df_dts_cln['RESULT'] = df_dts_cln['RESULT'].map(lambda x: cln_results(x))

In [100]:
df_dts_cln['RESULT'].unique()

array(['Negative', 'Positive', 'ms', nan], dtype=object)

In [101]:
df_dts_cln.head(2)

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,2021-01-01,2021-01-01,Negative,2021-01-01
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,2020-12-31,2021-01-01,Negative,2021-01-01


### **Remove Duplicates**

In [102]:
df_cln_srt = df_dts_cln.sort_values(['S_NUM', 'DT_CONF'])

In [103]:
df_cln_srt[df_cln_srt.duplicated(['S_NUM', 'DT_CONF'], keep=False) == True]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9947,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
11324,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
9948,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
11325,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
9949,COVC00710,45.0,M,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21472,COVC22894,35.0,M,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,2021-01-26,2021-01-26,Negative,2021-01-27
21471,COVC22895,36.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,2021-01-26,2021-01-26,Positive,2021-01-27
21473,COVC22895,36.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,2021-01-26,2021-01-26,Positive,2021-01-27
24925,KEM-21-03-94425,17.0,M,NaN,Kenya,Kiambu,Thika,No,NaN,NaN,"Cough, Runny nose",2021-03-04,NaT,Positive,2021-03-11


In [104]:
df_cln_srt[df_cln_srt.duplicated(keep=False) == True]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9947,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
11324,COVC00708,35.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
9948,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
11325,COVC00709,32.0,F,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
9949,COVC00710,45.0,M,NaN,Kenya,Laikipia,Laikipia east,No,NaN,Nanyuki Teaching and Referral Hospital,NaN,2020-06-09,2020-06-11,Negative,2020-06-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21472,COVC22894,35.0,M,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,2021-01-26,2021-01-26,Negative,2021-01-27
21471,COVC22895,36.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,2021-01-26,2021-01-26,Positive,2021-01-27
21473,COVC22895,36.0,F,NaN,Kenya,Nairobi,Nan,No,NaN,NaN,NaN,2021-01-26,2021-01-26,Positive,2021-01-27
24925,KEM-21-03-94425,17.0,M,NaN,Kenya,Kiambu,Thika,No,NaN,NaN,"Cough, Runny nose",2021-03-04,NaT,Positive,2021-03-11


In [105]:
df_dedud = df_cln_srt.drop_duplicates(['S_NUM', 'DT_CONF'], keep='first')

In [106]:
df_dedud.shape

(24833, 15)

### **Export Cleaned Data**

In [107]:
df_dedud.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-cln.xlsx', index=False)

In [108]:
df_dedud[df_dedud['RESULT'] == 'Positive'].to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-cln-pos.xlsx', index=False)

In [109]:
df_dedud[df_dedud['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9607,COVE20,39.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,2020-09-10,2020-09-10,Negative,2020-09-11
9608,COVE21,50.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,2020-09-10,2020-09-10,Negative,2020-09-11
24807,DRC-BKV-1,46.0,M,NaN,Democratic Republic of the Congo,Bukavu,Nan,No,NaN,NaN,FC;SB;CO;GW,2021-06-30,2021-08-02,Positive,NaT
24887,DRC-BKV-10,NaN,NaN,NaN,Democratic Republic of the Congo,Bukavu,Nan,No,NaN,NaN,NaN,NaT,2021-08-02,Positive,NaT
24890,DRC-BKV-100,73.0,F,NaN,Democratic Republic of the Congo,Bukavu,Nan,No,NaN,NaN,FC;SB;CO;GW,NaT,2021-08-02,Positive,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25176,NPHL-12780,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19
25180,NPHL1,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19
25181,NPHL2,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19
25182,NPHL3,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19


In [110]:
df_dedud[df_dedud['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9607,COVE20,39.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,2020-09-10,2020-09-10,Negative,2020-09-11
9608,COVE21,50.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,NaN,NaN,NaN,2020-09-10,2020-09-10,Negative,2020-09-11
24807,DRC-BKV-1,46.0,M,NaN,Democratic Republic of the Congo,Bukavu,Nan,No,NaN,NaN,FC;SB;CO;GW,2021-06-30,2021-08-02,Positive,NaT
24887,DRC-BKV-10,NaN,NaN,NaN,Democratic Republic of the Congo,Bukavu,Nan,No,NaN,NaN,NaN,NaT,2021-08-02,Positive,NaT
24890,DRC-BKV-100,73.0,F,NaN,Democratic Republic of the Congo,Bukavu,Nan,No,NaN,NaN,FC;SB;CO;GW,NaT,2021-08-02,Positive,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25176,NPHL-12780,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19
25180,NPHL1,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19
25181,NPHL2,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19
25182,NPHL3,NaN,NaN,NaN,Kenya,NaN,Nan,No,NaN,NaN,NaN,NaT,2021-04-07,Positive,2021-04-19


11-09-2020

COVC9774
COVC9775
COVE20
COVE21
COVC9776
COVC9777


### **ID Misssing Data**

In [111]:
start = 0
df_missing = pd.DataFrame(columns=cols_rename)
for num in df_dedud['S_NUM'].sort_values():
#     curr_row = df_dts_cln[df_dts_cln['S_NUM']]
    try:
        curr = int(num.lstrip('COVC').lstrip('0'))
        
        if curr - start > 1:
            start_id = f'COVC{start}'
            cov_id = f'COVC{curr}'
            df_missing = df_missing.append(df_dts_cln[df_dts_cln['S_NUM'] == cov_names(start_id)])
            df_missing = df_missing.append(df_dts_cln[df_dts_cln['S_NUM'] == cov_names(cov_id)])
        start = curr
    except ValueError:
#         print(f'ValueError: {num}')
        pass

In [112]:
df_missing.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [113]:
(df_missing[['S_NUM', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'DT_CONF']]
 .sort_values(['S_NUM', 'DT_CONF'])
 .drop_duplicates('S_NUM', keep='last')
 .to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-missingII.xlsx', index=False))

### **ID Missing Results**

In [114]:
(df_dts_cln[df_dts_cln['RESULT'].isin(['Positive', 'Negative']) == 
            False][['S_NUM', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'DT_CONF', 'RESULT']]
.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-results-missingII.xlsx', index=False))

### **Filter Samples Seq'd in seq10**

In [115]:
# df_waiting = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocsIII-flt.xlsx')
# df_ext_seq10 = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/run10_extractionplates.xlsx')

In [116]:
# df_waiting.head()

In [117]:
# df_ext_seq10.head()

In [118]:
# ext_list = list(df_ext_seq10['Sample_Name'])

In [119]:
# df_waiting11 = df_waiting[df_waiting['Sample_Name'].isin(ext_list) == False]

In [120]:
# df_waiting11.to_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocsIII-flt2.xlsx', index=False)

In [121]:
# plt.style.available

### **Metadata + Cts**

In [122]:
df_Cts = pd.read_excel('/home/douso/Documents/RunSheets/Cts/runsheet-cts.xlsx')[['Sample_Name', 'Well_Position', 'Ct_Mean']]
# df_nhrl_Cts = pd.read_excel('/home/douso/Documents/RunSheets/Cts/NHRL-cts.xlsx')
# df_Cts = df_covc_Cts.append(df_nhrl_Cts)

In [123]:
df_metCts = df_dedud.set_index('S_NUM').merge(df_Cts.set_index('Sample_Name'), how='outer', left_index=True, right_index=True).reset_index().rename(columns={'index': 'S_NUM'})

In [124]:
df_metCts.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF,Well_Position,Ct_Mean
0,COVC00001,35.0,F,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,2020-06-02,NaN,NaN
1,COVC00002,29.0,M,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,2020-06-02,NaN,NaN
2,COVC00003,50.0,M,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,2020-06-02,NaN,NaN
3,COVC00004,35.0,F,Casual,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,2020-06-02,NaN,NaN
4,COVC00005,27.0,F,Refugee,NaN,Garissa,Dadaab,No,NaN,NaN,NaN,NaT,NaT,Negative,2020-06-02,NaN,NaN


In [125]:
df_metCts.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-resultsCts-merged-cln.xlsx', index=False)

In [126]:
df_metCts.shape

(24865, 17)

In [127]:
df_metCts[df_metCts['RESULT'] == 'Positive'].to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-resultsCts-merged-cln-pos.xlsx', index=False)

In [128]:
dt = datetime.today().strftime(format='%d-%m-%Y')

#### **Confirm updated file in the following cell**

In [129]:
df_sequenced_samples = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/all-sequenced-samples-IDs_07-09-2021.xlsx')
seqd_list = list(df_sequenced_samples['SAMPLE'])

In [130]:
df_dedud_pos = df_metCts[df_metCts['RESULT'] == 'Positive']
df_dedud_pos_unseq = df_dedud_pos[df_dedud_pos['S_NUM'].isin(seqd_list) == False]
df_dedud_pos_unseq.to_excel(f'/home/douso/Documents/TrendData/Results/ResultsMerged/all-unsequenced-pos-samples_{dt}.xlsx', index=False)

### Positives 2021 - Unsequenced

In [131]:
df_dedud_pos_unseq_21 = df_dedud_pos_unseq[df_dedud_pos_unseq['DT_CONF'] >= pd.to_datetime('2021-01-01 00:00:00')]
df_dedud_pos_unseq_21.to_excel(f'/home/douso/Documents/TrendData/Results/ResultsMerged/all-unsequenced-pos-samples-2021_{dt}.xlsx', index=False)